# Notebook #1 for AHED Project
___
We have a spreadsheet of 3 sheets. Each sheet contains publications related to the Astrobiology Habitable Environments Database (AHED). The question we would like to answer is: which of these publications are currently not indexed in our database? 
___

## Task 1: Match AHED to ADS Items by DOI
Create a version of this spreadsheet that has a column "bibcode" added to the right of the DOI column. For those publications we are able to match to ADS records, this column with list these bibcodes, otherwise "NA".

Outline:
- Step 1: Data Cleanup and Prep - combine all the excel sheets to one data set
- Step 2: Isolate/create list of DOIs
- Step 3: API Connection & Query 
- Step 4: Match list of response bibcodes to original data set
___

### Step 1: Data Cleanup and Prep
Reading and concatenating the excel files, and then cleaning up the data to make sure it's usable for querying the API.

In [1]:
import pandas as pd

# Read excel sheets
astro_pubs = pd.read_excel("/Users/sao/Documents/Python-Projects/AHED/SpaceScienceAndAstrobiologyDivision.xlsx", sheet_name=0)
planet_pubs = pd.read_excel("/Users/sao/Documents/Python-Projects/AHED/SpaceScienceAndAstrobiologyDivision.xlsx", sheet_name=1)
exo_pubs = pd.read_excel("/Users/sao/Documents/Python-Projects/AHED/SpaceScienceAndAstrobiologyDivision.xlsx", sheet_name=2)

# Combine the excel sheets into one new/big data frame
# 862 total papers
ahed_pubs = pd.concat([astro_pubs, planet_pubs, exo_pubs], axis='index', ignore_index=True)

# Export to new excel file
ahed_pubs.to_excel("AHED/ahed_pubs.xlsx", index=False)
ahed_pubs

,AUTHOR (S),ORG,TITLE OF PAPER,JOURNAL,YEAR,DOI
0,"Cartwright, Richard J. search by orcid ; Bedd...",SSA,The Science Case for Spacecraft Exploration of...,"The Planetary Science Journal, Volume 2, Issue...",2021,10.3847/PSJ/abfe12
1,"MacKenzie, Shannon M. search by orcid ; Birch...",SSA,"Titan: Earth-like on the Outside, Ocean World ...","The Planetary Science Journal, Volume 2, Issue...",2021,10.3847/PSJ/abf7c9
2,"Scott A. Sandford, Donald E. Brownlee, Michael...",SSA,The Stardust Sample Return Mission (Book Chapt...,Sample Return Missions: The Last Frontier of S...,2021,https://www.elsevier.com/books/sample-return-m...
3,"Crossfield, Ian J. M. search by orcid ; Dress...",SSA,K2-138 g: Spitzer Spots a Sixth Planet for the...,"The Astronomical Journal, Volume 161, Issue 5,...",2021,10.3847/1538-3881/abeab0
4,"Scipioni, F. ; White, O. ; Cook, J. C. ; Bert...",SSA,Pluto's Sputnik Planitia: Composition of geolo...,"Icarus, Volume 359, article id. 114303.",2021,10.1016/j.icarus.2021.114303
...,...,...,...,...,...,...
857,Tori M. Hoehler & Bo Barker Jørgensen,SSX,Microbial life under extreme energy limitation,doi:10.1038/nrmicro2939,2013,NaN
858,"Pohorille, A.",SSX,Molecular Dynamics Simulation of the Antiamoeb...,"Biophys. J. 100, 2394-2402",2013,NaN
859,"Des Marais, D. J.",SSX,Planetary climate and the search for life. In ...,The University of Arizona Press,2013,NaN
860,Europa Lander Science Definition Team includin...,SSX,Science potential from a Europa Lander,"Astrobiology 13, 740-773",2013,NaN


### Step 2: Create DOI list
Cleaned up the ahed_pubs DOIs in OpenRefine:
- Removed extra characters like 'doi:' and 'doi.org/'
- Renamed the column headers
- Duplicates: Removed any duplicates i could find
- Outliers: I removed the 'Elsevier' url, and the 'ERRATUM' link. (I will have to manually review those two later.)
- Cleaned up publication/journal info to use for reference service/resolver

I saved this as a new excel, 'ahed_pubs_refined' with total 797 papers.

In [2]:
# Import new excel sheet, 'ahed_pubs_refined'
ahed_pubs_refined = pd.read_excel("AHED/ahed_pubs_refined.xlsx")

# Isolate the DOIs and drop all the papers that have no DOIs (drop null values)
ahed_dois = ahed_pubs_refined['DOI']
ahed_dois.dropna(inplace = True)

# Convert it from a data frame to a list
ahed_doi_list = ahed_dois.to_list()
print("Original paper list has", len(ahed_doi_list), "DOIs to search.")

Original paper list has 177 DOIs to search.


### Step 3: API Connection
Run the DOI list against the ADS API, and return the bibcodes.

In [8]:
import requests
import json

# --- API REQUEST --- 
token = "pHazHxvHjPVPAcotvj7DIijROZXUjG5vXa2OaCQO"
url = "https://api.adsabs.harvard.edu/v1/search/query?"

data=[]

for i in range(0, len(ahed_doi_list), 20):
    chunk = ahed_doi_list[i:i + 20]
    tagged = ['doi:' + d for d in chunk]
    query = ' OR '.join(tagged)
    
    params = {"q":query,"fl":"doi,bibcode","rows":200}
    headers = {'Authorization': 'Bearer ' + token}
    response = requests.get(url, params=params, headers=headers)
#     print(data.text, '\n')

    from_solr = response.json()
    if (from_solr.get('response')):
        num_docs = from_solr['response'].get('numFound', 0)
        if num_docs > 0:
            for doc in from_solr['response']['docs']:
                data.append((doc['bibcode'],doc['doi'][0]))
#     print(data)

dois_matched = pd.DataFrame(data, columns = ['bibcode','DOI'])

# Export new excel sheet
dois_matched.to_excel("AHED/dois_matched.xlsx",
                  index=False)

dois_matched

,bibcode,DOI
0,2014Sci...343A.386G,10.1126/science.1242777
1,2020Sci...370.3557K,10.1126/science.abc3557
2,2014AsBio..14..534D,10.1089/ast.2014.1150
3,2020A&A...642A..49D,10.1051/0004-6361/202038616
4,2014IAUS..297..364S,10.1017/S174392131301613X
...,...,...
150,2014AsBio..14..577S,10.1089/ast.2013.1131
151,2021PSJ.....2..112M,10.3847/PSJ/abf7c9
152,2014IAUS..297..103C,10.1017/S1743921313015664
153,2021PSJ.....2..120C,10.3847/PSJ/abfe12


### Step 4: Match DOI/Bibcode Response Data to Original Paper List

In [10]:
# Merge/Join new table to original, joined on 'DOI'
merged = ahed_pubs_refined.merge(dois_matched, on='DOI', how='left')

# Count my running total of bibcodes matched
# merged = merged.dropna(subset=['BIBCODE'])

# Export merged data
merged.to_excel("AHED/dois_matched.xlsx",
                  index=False)

merged

,AUTHORS,ORG,TITLE,JOURNAL,YEAR,DOI,bibcode
0,"Misra, Anupam K., Acosta-Maeda, Tayro E., Shar...",SST,"Standoff Biofinder for Fast, Noncontact, Nonde...","AsBio, 16, 9, 715-729",2016,NaN,NaN
1,"Chilcote, Jeffrey, Pueyo, Laurent, De Rosa, Ro...",SST,1-2.4 μm Near-IR Spectrum of the Giant Planet ...,"AJ, 153",2017,NaN,NaN
2,"X. Huang, D. W. Schwenke, and T. J. Lee,",SST,212. Empirical InfraRed Line Lists for Five SO...,"JMoSp, 311, 19",2015,NaN,NaN
3,"230. R. C. Fortenberry, J. S. Francisco, and T...",SST,230. Towards the Astronomical Detection of the...,"ApJ, 819, 141",2016,NaN,NaN
4,"Everett, Mark, Fridlund, Malcolm, Fukui, Akihi...",SSA,44 Validated Planets from K2 Campaign 10,"AJ, 156, 2, 78",2018,NaN,NaN
...,...,...,...,...,...,...,...
792,"D. L. Bish, D. F. Blake, D. T. Vaniman, S. J. ...",SSX,X-Ray Diffraction Results from Mars Science La...,"Sci, 341, 1238932",2013,NaN,NaN
793,"Cox, N.L.J., Cami, J., Kaper, L., Foing, B. H....",SSA,X-Shooter Survey of Near-Infrared DIBs,"IAU, 9",2013,10.1017/S1743921313015664,2014IAUS..297..103C
794,"Curtis, Jason L., Douglas, Stephanie T., Esque...",SSA,Zodiacal Exoplanets in Time (ZEIT). VII. A Tem...,"AJ, 156, 2, 46",2018,NaN,NaN
795,"Apai, D., Karalidi, T., Marley, M. S., Yang, H...",SST,"Zones, spots, and planetary-scale waves beatin...","Sci, 357",2017,NaN,NaN


Status/Summary:
- We started with 862 papers from the provided AHED spreadsheets
- Refined it to 797 papers removing duplicates
- Extracted 177 DOIs
- Matched 156 DOIs to existing ADS Bibcodes